### Drugi deo - klasifikacija planeta i satelita uz veštačku augmentaciju slika

Kao pokušaj poboljšanja efikasnosti modela, augmentacija slika može da pomogne. Tako će se baza slika proširiti i model će imati veći skup na raspolaganju za učenje, validaciju i treniranje, i bolju sposobnost prilagođavaja na neviđenim podacima.


Biće primenjen isti model kao i u prvom delu, kako na planetama tako i na satelitima, radi upoređivanja konačnih dobijenih rezultata.

## Planete

Za učitavanje slika, ovaj put će biti prikazano korišćenje klase ImageDataGenerator.

Klasa *ImageDataGenerator* u Kerasu omogućava različite tehnike augmentacije podataka, poput rotacije, skaliranja, horizontalnog i vertikalnog zrcaljenja, promene svetline i kontrasta, što pomaže u obogaćivanju skupa podataka i poboljšanju generalizacije modela.  

ImageDataGenerator se obično koristi zajedno sa metodama *flow_from_directory* ili *flow* za generisanje podataka u obliku koji je pogodan za treniranje modela.

##### Učitavanje potrebnih biblioteka

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import random
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten
from tensorflow.keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

2024-03-11 16:05:10.764936: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 16:05:10.954821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 16:05:10.954875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 16:05:10.989124: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 16:05:11.081422: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 16:05:11.081791: I tensorflow/core/platform/cpu_feature_guard.cc:1

I ovog puta, baza slika će biti kreirana pomoću putanja do njih.

In [13]:
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Koreni direktorijum u kom se nalaze slike
data_dir = 'planete'

# Definisanje imena klasa u alfabetskom redu
planete = ['Jupiter', 'Mars', 'Merkur', 'Neptun', 'Saturn', 'Uran', 'Venera', 'Zemlja']

# Definisanje praznih listi u koje će biti smeštene putanje i oznake klasa
putanje = []
oznake = []

# Popunjavnje listi
for i, planeta in enumerate(planete):
    planeta_dir = os.path.join(data_dir, planeta)
    fajlovi = os.listdir(planeta_dir)
    putanje += [os.path.join(planeta_dir, fajl) for fajl in fajlovi]
    oznake += [i] * len(fajlovi)

# Stratifikovana podela skupa na test i trening, 70% za trening, 30% za test
# Random_state parametar je tu da bi podela pri svakom pokretanju bila istovetna
X_train, X_test, y_train, y_test = train_test_split(putanje, oznake, test_size=0.3, stratify=oznake, random_state=42)

# Konvertovanje oznaka, koje su trenutno tipa int, u stringove, da bi model mogao da ih primi
y_train = [str(oznaka) for oznaka in y_train]
y_test = [str(oznaka) for oznaka in y_test]

dataframe = pd.DataFrame({'filename':X_train, 'class':y_train})

# Korišćenje klase ImageDataGeneratori definisanje augmentacija koje će biti primenjene na slike
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalizacija piksela na [0, 1]
    rotation_range=20,  # Nasumična rotacija slika do 20 stepeni
    width_shift_range=0.2,  # Nasumično pomeranje slika horizontalno do 20% širine
    height_shift_range=0.2,  # Nasumično pomeranje slika vertikalno do 20% visine
    shear_range=0.2,  # Ugao pomeranja u radijanima
    zoom_range=0.2,  # Nasumično uvećanje slika do 20% 
    horizontal_flip=True,  # Nasumično horizontalno obrtanje slika
    fill_mode='nearest'  # U slučaju nedostajućih piksela, popuniti vrednosti koristeći najbližu vrednost
)

# Definisanje veličine gomile i veličine slike
batch_size = 32
target_size = (256, 256) 

# Generatori za trening i test skup
train_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_train, 'class': y_train}),
    directory=None,
    x_col='filename',
    y_col='class',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse', 
    interpolation="lanczos",
    shuffle=True
)

test_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_test, 'class': y_test}),
    directory=None,
    x_col='filename',
    y_col='class',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse',
    interpolation="lanczos",
    shuffle=False
)

Found 834 validated image filenames belonging to 8 classes.
Found 358 validated image filenames belonging to 8 classes.


##### Učitavanje prethodno napravljenog modela

In [9]:
from tensorflow.keras.models import load_model

model = load_model('modelP')

### Kompilacija, treniranje i evaluacija modela

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, batch_size=batch_size, epochs=20, verbose=1)

evaluacija = model.evaluate(test_generator)
print("Preciznost na test skupu:", evaluacija[1])
print("Gubitak na test skupu:", evaluacija[0])


Epoch 1/20
27/27 [==============================] - 41s 1s/step - loss: 0.1533 - accuracy: 0.9484
Epoch 2/20
27/27 [==============================] - 40s 1s/step - loss: 0.1247 - accuracy: 0.9436
Epoch 3/20
27/27 [==============================] - 37s 1s/step - loss: 0.1117 - accuracy: 0.9652
Epoch 4/20
27/27 [==============================] - 36s 1s/step - loss: 0.0861 - accuracy: 0.9676
Epoch 5/20
27/27 [==============================] - 37s 1s/step - loss: 0.0995 - accuracy: 0.9724
Epoch 6/20
27/27 [==============================] - 37s 1s/step - loss: 0.0676 - accuracy: 0.9772
Epoch 7/20
27/27 [==============================] - 37s 1s/step - loss: 0.1220 - accuracy: 0.9748
Epoch 8/20
27/27 [==============================] - 40s 1s/step - loss: 0.0888 - accuracy: 0.9700
Epoch 9/20
27/27 [==============================] - 39s 1s/step - loss: 0.0527 - accuracy: 0.9796
Epoch 10/20
27/27 [==============================] - 36s 1s/step - loss: 0.0434 - accuracy: 0.9868
Epoch 11/20
27/27 [